In [2]:
import oracledb as od
import pandas as pd
import numpy as np
import config
from tqdm import tqdm
import ast
import re

In [3]:
def load_recipe(n =1000):
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12") # db connection
    conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],  dsn = config.DB_CONFIG['dsn'])
    exe = conn.cursor()
    exe.execute(f'select * from recipe_table where rownum <= {n}')
    row = exe.fetchall() # row 불러오기
    column_name = exe.description # column 불러오기
    columns=[]
    for i in column_name:
        columns.append(i[0])
    result = pd.DataFrame(row, columns=columns) # row, column을 pandas DataFrame으로 나타내기
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    conn.close()
    return result

In [4]:
raw_data = load_recipe(n=1000)

In [5]:
raw_data

,recipe_url,recipe_title,recipe_summary1,recipe_summary2,recipe_summary3,recipe_ingredients,recipe_step,recipe_tag,recipe_comment,recipe_view,recipe_category_type,recipe_category_method,recipe_category_situation
0,http://www.10000recipe.com/recipe/6892567,만물상 5분 콩국수,1인분,60분 이내,초급,"{'재료': ['메주콩5소주잔', '옥수수알1소주잔', '물2컵', '중면1인분',...",['#1 메주를 만들 때 사용하는 콩이라 메주콩이라고도 불리죠. 이 콩이 소주잔으로...,"['콩국수', '국수', '여름요리', '만물상', '만물상5분콩국수', '제철요리...",1,21000,면/만두,삶기,일상
1,http://www.10000recipe.com/recipe/6868141,아삭하고 깔끔한 오이소박이 비법 *^^*,4인분,60분 이내,초급,"{'재료': ['오이10개', '부추2/3단', '쪽파6줄기', '양파1개', '천...",['#1 1. 오이는 10개 준비합니다. 오이를 씻을때는 천일염으로 오이 표면을 비...,"['오소박이', '오이소박이 레시피', '오이소박이 담그는법']",0,12000,김치/젓갈/장류,절임,일상
2,http://www.10000recipe.com/recipe/6862068,탕수육...바삭한 탕수육에 특제 탕수육소스^^,4인분,60분 이내,초급,"{'재료': ['돼지고기안심400g', '양파1/2개', '표고버섯(목이버섯)4개'...","['#1 향신채>마늘은 편썰기,대파는 흰부분만 1대를 길게 + 십자형 컷팅을 해서 ...","['탕수육', '돼지안심', '돼지고기요리', '카라멜탕수육', '돼지탕수육']",0,53000,양념/소스/잼,튀김,일상
3,http://www.10000recipe.com/recipe/6789480,잔멸치나 새우보다 칼슘이 최고! 견과류를 넣은 뱅어포조림,2인분,30분 이내,초급,"{'재료': ['뱅어포3장', '식용유1/2컵', '아몬드20알', '해바라기씨1큰...","['#1 뱅어포를 준비하시구요.', '#2 가위로 잘라주면 편한데, 먼저 2등분을 ...","['견과류', '뱅어포', '조림', '뱅어포조림', '철분이 많은', '빈혈', ...",3,9418,밑반찬,조림,일상
4,http://www.10000recipe.com/recipe/6879029,한우 모듬 초밥,1인분,30분 이내,아무나,"{'재료': ['한우차돌박이1/2줌', '노란파프리카1/4개', '빨간파프리카1/4...","['#1 재료 준비!', '#2 파프리카와 양파는 채 썰어주세요.', '#3 팽이버...","['초밥', '차돌박이', '소고기', '술안주', '안주', '저녁']",0,3946,밥/죽/떡,굽기,술안주
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,http://www.10000recipe.com/recipe/6883584,초장 황금레시피,2인분,5분 이내,아무나,"{'재료': ['고추창2큰술', '매실액1큰술', '설탕1큰술', '식초1.5큰술'...","['#1 모든재료 아빠수저 기준입니다!고추장 2큰술', '#2 매실액 1큰술도 넣어...",[],13,111000,양념/소스/잼,무침,초스피드
996,http://www.10000recipe.com/recipe/6878123,면역력 높이는 단짠 반찬/단호박 견과류 꿀조림,3인분,30분 이내,초급,"{'재료': ['단호박1개', '견과류1줌', '꿀3스푼', '간장1스푼', '기름...","['#1 재료를 준비해 주세요~', '#2 단호박은 깨끗이 씻은 후 반으로 잘라 속...","['단호박조림', '가을제철요리', '단호박꿀조림', '단호박 견과류조림', '단호...",0,4791,메인반찬,조림,영양식
997,http://www.10000recipe.com/recipe/6872084,흑마늘넣고 조린 코다리조림,4인분,60분 이내,아무나,"{'재료': ['코다리900g', '진간장100ml', '올리고당150ml', '매...",['#1 냉동 코다리는 지느러미와 아가미 부분은 가위를 이용해 정리하고 ( 먹기도 ...,[],0,889,메인반찬,조림,영양식
998,http://www.10000recipe.com/recipe/1952659,장어간장강정,4인분,60분 이내,초급,"{'재료': ['붕장어2마리', '노랑파프리카1/3개', '빨강파프리카1/3개', ...",['#1 간장소스 재료를 모두 넣고 약한불에서 졸여주다가 10분후 다시마는 건져내주...,[],0,6410,메인반찬,튀김,영양식


In [6]:
def recipe_preprocessing(raw):
    data = raw.loc[raw['recipe_ingredients'].notnull()].copy()  # None 값 제거

    def clean_ingredients(ingredients):
        if ingredients is not None:
            ingredients = ingredients.replace('\\ufeff', '').replace('\\u200b', '')
        return ingredients
    data["recipe_ingredients"] = data["recipe_ingredients"].apply(clean_ingredients)
    result = data[['recipe_title', 'recipe_ingredients']]

    return result

In [7]:
recipe = recipe_preprocessing(raw_data)
recipe

,recipe_title,recipe_ingredients
0,만물상 5분 콩국수,"{'재료': ['메주콩5소주잔', '옥수수알1소주잔', '물2컵', '중면1인분',..."
1,아삭하고 깔끔한 오이소박이 비법 *^^*,"{'재료': ['오이10개', '부추2/3단', '쪽파6줄기', '양파1개', '천..."
2,탕수육...바삭한 탕수육에 특제 탕수육소스^^,"{'재료': ['돼지고기안심400g', '양파1/2개', '표고버섯(목이버섯)4개'..."
3,잔멸치나 새우보다 칼슘이 최고! 견과류를 넣은 뱅어포조림,"{'재료': ['뱅어포3장', '식용유1/2컵', '아몬드20알', '해바라기씨1큰..."
4,한우 모듬 초밥,"{'재료': ['한우차돌박이1/2줌', '노란파프리카1/4개', '빨간파프리카1/4..."
...,...,...
995,초장 황금레시피,"{'재료': ['고추창2큰술', '매실액1큰술', '설탕1큰술', '식초1.5큰술'..."
996,면역력 높이는 단짠 반찬/단호박 견과류 꿀조림,"{'재료': ['단호박1개', '견과류1줌', '꿀3스푼', '간장1스푼', '기름..."
997,흑마늘넣고 조린 코다리조림,"{'재료': ['코다리900g', '진간장100ml', '올리고당150ml', '매..."
998,장어간장강정,"{'재료': ['붕장어2마리', '노랑파프리카1/3개', '빨강파프리카1/3개', ..."


In [8]:
#1. 식재료 단위 별로 쪼개기
def split_ingredient(data):
    for i in range(1, 21):
        data.loc[:, f'ingredient{i}'] = None
        data.loc[:, f'quantity{i}'] = None
        data.loc[:, f'unit{i}'] = None
   


    # 패턴과 일치하지 않는 데이터를 저장할 딕셔너리
    non_matching_items = {}
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]): #tqdm으로 진행상황 확인
        if row['recipe_ingredients']:
            ingredients_dict = ast.literal_eval(row["recipe_ingredients"]) #딕셔너리 형태로 저장된 recipe_ingredients 불러오기
            ingredient_count = 1
            for category, items in ingredients_dict.items(): #category : 재료, 양념재료, items: 사과1개, 돼지고기600g
                if items:  # 아이템이 존재하는 경우
                    for item in items:
                        match = re.match(r'([가-힣a-zA-Z]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item) # 정규식
                        if match:
                            ingredient, _, quantity, unit = match.groups()
                            
                            data.loc[idx, f'ingredient{ingredient_count}'] = ingredient
                            data.loc[idx, f'quantity{ingredient_count}'] = quantity
                            data.loc[idx, f'unit{ingredient_count}'] = unit

                            ingredient_count += 1
                        else:
                            # 패턴과 일치하지 않는 경우 딕셔너리에 추가
                            non_matching_items[idx] = item
        else:
            pass

    # 패턴과 일치하지 않는 데이터 출력
    for idx, item in non_matching_items.items():
        print(f'Row {idx}: {item}')
    return data

In [12]:
new_recipe = split_ingredient(recipe)
new_recipe

100%|██████████| 1000/1000 [00:02<00:00, 454.14it/s]

Row 18: 2배식초약간
Row 38: 
Row 56: 
Row 75: 
Row 91: 
Row 120: (새송이)버섯
Row 156: 0.5cm두께로자른스팸4조각
Row 163: 
Row 213: 1/4작은술
Row 301: 
Row 333: .냉동꽃게2kg
Row 374: 
Row 421: (말탈리아티)파스타면적당량
Row 513: 
Row 532: 2배식초1/2종이컵
Row 555: 3층치즈2개
Row 738: 
Row 783: 3층치즈2장
Row 802: 10무1단


,recipe_title,recipe_ingredients,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,...,unit71,ingredient72,quantity72,unit72,ingredient73,quantity73,unit73,ingredient74,quantity74,unit74
0,만물상 5분 콩국수,"{'재료': ['메주콩5소주잔', '옥수수알1소주잔', '물2컵', '중면1인분',...",메주콩,5,소주잔,옥수수알,1,소주잔,물,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,아삭하고 깔끔한 오이소박이 비법 *^^*,"{'재료': ['오이10개', '부추2/3단', '쪽파6줄기', '양파1개', '천...",오이,10,개,부추,2/3,단,쪽파,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,탕수육...바삭한 탕수육에 특제 탕수육소스^^,"{'재료': ['돼지고기안심400g', '양파1/2개', '표고버섯(목이버섯)4개'...",돼지고기안심,400,g,양파,1/2,개,표고버섯(목이버섯),4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,잔멸치나 새우보다 칼슘이 최고! 견과류를 넣은 뱅어포조림,"{'재료': ['뱅어포3장', '식용유1/2컵', '아몬드20알', '해바라기씨1큰...",뱅어포,3,장,식용유,1/2,컵,아몬드,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,한우 모듬 초밥,"{'재료': ['한우차돌박이1/2줌', '노란파프리카1/4개', '빨간파프리카1/4...",한우차돌박이,1/2,줌,노란파프리카,1/4,개,빨간파프리카,1/4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,초장 황금레시피,"{'재료': ['고추창2큰술', '매실액1큰술', '설탕1큰술', '식초1.5큰술'...",고추창,2,큰술,매실액,1,큰술,설탕,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,면역력 높이는 단짠 반찬/단호박 견과류 꿀조림,"{'재료': ['단호박1개', '견과류1줌', '꿀3스푼', '간장1스푼', '기름...",단호박,1,개,견과류,1,줌,꿀,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,흑마늘넣고 조린 코다리조림,"{'재료': ['코다리900g', '진간장100ml', '올리고당150ml', '매...",코다리,900,g,진간장,100,ml,올리고당,150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,장어간장강정,"{'재료': ['붕장어2마리', '노랑파프리카1/3개', '빨강파프리카1/3개', ...",붕장어,2,마리,노랑파프리카,1/3,개,빨강파프리카,1/3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def recipe_food_matrix(data):
    def convert_fraction_to_float(quantity):
        from fractions import Fraction

        try:
            return float(Fraction(quantity))
        except ValueError:
            return None 
    def convert_unit_to_number(unit):
        '''
        단위에 따른 g 수 변환
        '''
        unit_conversion = {
            'g': 1,
            '개': 100,
            '조금' :10
        }
        return unit_conversion.get(unit, 1)
    ingredient_columns = data.filter(like='ingredient').drop(columns=['recipe_ingredients'])
    all_ingredients = [item for sublist in ingredient_columns.values for item in sublist if pd.notna(item)] 
    all_ingredients = set()
    for i in range(1, 21):  
        all_ingredients.update(data[f'ingredient{i}'].dropna().unique())

    recipe_ingredients_df = pd.DataFrame(columns=list(all_ingredients))

    recipe_rows = []
    for idx, row in data.iterrows():
        recipe_data = {ingredient: 0.0 for ingredient in all_ingredients}  # 모든 식재료를 None으로 초기화
        for i in range(1, 21):  
            ingredient = row[f'ingredient{i}']
            quantity = row[f'quantity{i}']
            unit = row[f'unit{i}']
            if pd.notna(ingredient) and pd.notna(quantity):
                quantity_float = convert_fraction_to_float(quantity)
                if quantity_float is not None:
                    unit_number = convert_unit_to_number(unit) if pd.notna(unit) else 1
                    recipe_data[ingredient] = quantity_float * unit_number
        recipe_rows.append(recipe_data)

    # 새로운 데이터프레임 생성 (모든 식재료를 열로 가짐)
    recipe_ingredients_df = pd.concat([pd.DataFrame([row]) for row in recipe_rows], ignore_index=True)

    recipe_ingredients_df = recipe_ingredients_df.astype('float64')

    return recipe_ingredients_df

In [13]:
new_recipe = recipe_food_matrix(new_recipe)
new_recipe

,오리스테이크,매운고추,샌드위치햄,살라미,오일,민트잎약간,진간장또는맛간장,단호박분말,다진두부,새우살,...,쌀조청,곰취,편마늘,돼지등뼈(감자탕용),달걀후라이,자몽,당근(대),쌈장약간,달래,동태
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# oracle db의 열의 최대갯수 = 1000개
# 자를 필요가 있음
new_recipe1 = new_recipe.iloc[:, 0:999]
new_recipe1 = new_recipe1.astype('str')
new_recipe1


,오리스테이크,매운고추,샌드위치햄,살라미,오일,민트잎약간,진간장또는맛간장,단호박분말,다진두부,새우살,...,미향,칠리파우더약간,완두콩,후춧가루톡톡톡,국수동전크기,샐러리,양송이버섯적당량,감자중간크기,파뿌리,간돼지고기
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
table_name = 'test3_table'
columns_info = ', '.join([f"{col} VARCHAR2(50)" for col in new_recipe1.columns])
columns_info

'오리스테이크 VARCHAR2(50), 매운고추 VARCHAR2(50), 샌드위치햄 VARCHAR2(50), 살라미 VARCHAR2(50), 오일 VARCHAR2(50), 민트잎약간 VARCHAR2(50), 진간장또는맛간장 VARCHAR2(50), 단호박분말 VARCHAR2(50), 다진두부 VARCHAR2(50), 새우살 VARCHAR2(50), 원당또는설탕 VARCHAR2(50), 콜라비 VARCHAR2(50), 채썬채소 VARCHAR2(50), 고기시즈닝 VARCHAR2(50), 액젖 VARCHAR2(50), 후추가루약간 VARCHAR2(50), 마늘없어도됨 VARCHAR2(50), 쑥갓약간 VARCHAR2(50), 호밀빵 VARCHAR2(50), 각종허브약간 VARCHAR2(50), 육수(물) VARCHAR2(50), 꽃소금작은술 VARCHAR2(50), 새송이버섯 VARCHAR2(50), 무순 VARCHAR2(50), 파스타 VARCHAR2(50), 소불고기 VARCHAR2(50), 노란프리카 VARCHAR2(50), 냉동만두한접시 VARCHAR2(50), 피자치즈작은거 VARCHAR2(50), 영양부추약간 VARCHAR2(50), 홍합살 VARCHAR2(50), 검은콩가루 VARCHAR2(50), 쪽파잎부분 VARCHAR2(50), 초간장약간 VARCHAR2(50), 크러쉬드레드페퍼 VARCHAR2(50), 다크초콜릿 VARCHAR2(50), 갖가지나물잘게자라 VARCHAR2(50), 다시마육수적당량 VARCHAR2(50), 올리브오일약간 VARCHAR2(50), 스팸 VARCHAR2(50), 머스터드소스 VARCHAR2(50), 샐러드채소 VARCHAR2(50), 물엿 VARCHAR2(50), 크림치즈 VARCHAR2(50), 과일쨈 VARCHAR2(50), 밤 VARCHAR2(50), 물넉넉하게 VARCHAR2(50), 이스트슈가약간 VARCHAR2(50), 당근채썰어서 VARCHAR2(50), 이탈리안시즈닝 VARCHAR2(50), 참치 VAR

In [37]:
# oracle 테이블 생성

import config
import oracledb as od
# db connection
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                    dsn = config.DB_CONFIG['dsn'])


# 테이블 이름 및 컬럼 정보 추출
table_name = 'test3_table'
columns_info = ', '.join([f'"{col}" VARCHAR2(50)' for col in new_recipe1.columns])

# 테이블 생성
create_table_sql = f"CREATE TABLE {table_name} ({columns_info})"

exe = conn.cursor()
exe.execute(create_table_sql)
exe.close()

In [38]:
# oracle db에 insert 
def insert_into_oracle(dataframe, table_name, connection):    
    exe = conn.cursor()    
    # # 컬럼 수에 따라 바인드 변수 생성
    # bind_vars = ','.join([':' + str(i+1) for i in range(min(999, len(dataframe.columns)))])  # 처음 10개의 컬럼만 사용

    # # Pandas DataFrame의 데이터를 튜플로 변환
    # data_to_insert = [tuple(row[:999]) for _, row in dataframe.iterrows()]  # 처음 10개의 컬럼만 사용

    # # executemany를 사용하여 여러 행을 한 번에 삽입
    # exe.executemany(f"INSERT INTO {table_name} VALUES ({bind_vars})", data_to_insert)
    insert = [tuple(x) for x in dataframe.values]
    exe.executemany(
        f"INSERT INTO {table_name} VALUES ({','.join([':' + str(i+1) for i in range(len(dataframe.columns))])})",
        insert)
    
    connection.commit()
    exe.close()
    
    
insert_into_oracle(new_recipe1, table_name, conn)
conn.close()